In [94]:
# COMANDO PARA COMPACTAR O ARQUIVO EM GZIP PARA O MODELO

# for arquivo in *; do gzip -c "$arquivo" > "${arquivo}.gz"; done

# ==================== ==================== ====================

In [95]:
#==========CARREGANDO FUNCOES  #========================================================================
#========================================================================
# IMPORTANDO PACOTES ====
import os 
import warnings
import numpy as np
import pandas as pd 
import import_ipynb
import datetime as dt
warnings.filterwarnings('ignore')
from datetime import datetime
#from FuncaoProposta1 import retorna_lista
#import FuncaoProposta as fpl   # NECESSITA INSTALAR OUTROS PROGRAMAS
#====== FUNCOES  
def busca_data(modelo_inpe, media_gerada, caminho_salvar, nome_do_dado):
    #pegando tamanho das listas
    tam_amv_inpe = len(modelo_inpe)
    tam_media_gerada = len(media_gerada)
    
    # fazendo laço para buscar nome ###### ALTERAR AQUI PARA DEFINIR A QUANTIDADE DE VEZES
    for i in range(0,tam_amv_inpe,1): #tam_amv_inpe
        for j in range(0,tam_media_gerada,1):
            data_model_inpe = retorna_data_modelo_inpe(modelo_inpe[i])
            data_media_gerada = retorna_data__media_gerada(media_gerada[j]) 
            
            if( data_model_inpe == data_media_gerada ):
                # CHAMANDO A FUNCAO QUE ABRE O CSVS -
                #DENTRO DESSA FUNCAO CHAMA O SUBSTITUI E ARMAZENA
                print(f'{data_model_inpe} {data_media_gerada}\n')
                funcao_abre_csvs(modelo_inpe[i],
                                 media_gerada[j],
                                 caminho_salvar, nome_do_dado)
            else:
                #print(f'a data do csv inpe {data_model_inpe}')
                continue

    print("fim")
def funcao_abre_csvs(modelo_inpe,media_gerada,caminho_salvar, nome_do_dado):
    #abrindo dados csv inpe        
    dado_inpe = pd.read_csv( modelo_inpe,sep = ' ', skiprows =7,
                             header = 0, encoding = 'latin1'  )
    dado_inpe = pd.DataFrame(dado_inpe)
    #ABRINDO DADOS DAS MEDIAS QUE CALCULEI
    dado_medias = pd.read_csv( media_gerada,sep = ',', skiprows = 0,                        
                        header = 0, encoding = 'latin1'  )
    dado_medias = pd.DataFrame(dado_medias)
    
    #pegando titulo do documento
    titulo = modelo_inpe.split("/")[-1]
    
    # chamando funcao que substitui os arquivos
    teste = substitui_temps(dado_inpe, dado_medias, nome_do_dado )
    #================== SALVANDO OS ARQUIVOS EM CSV ===========
    teste.to_csv(f'{caminho_salvar}{titulo}',index = False, sep =' ', header = False)
    
    arquivo2 = reabre_arquivos(caminho_salvar, titulo)
    
    #pegando linhas iniciais
    linhas_iniciais = pegando_8linhas(modelo_inpe)
    
    #     Criando arquivo novo
    merge_arquivos(linhas_iniciais, arquivo2, caminho_salvar, titulo)
    
    
    print(f'{titulo}, salvo em {caminho_salvar}')
def retorna_data__media_gerada(caminho_media_gerada):
    aux = caminho_media_gerada.split('_')[-3]
    ano= int(aux[1:5])
    dj = int(aux[5:8])
    hora= int(aux[8:10])
    data_amv = datetime.strptime(f"{ano}-{dj}", "%Y-%j")  # Convertendo ano e dia juliano
    data_amv = data_amv.replace(hour=hora)
    return data_amv
def retorna_data_modelo_inpe(caminho_modelo_inpe):
    aux = caminho_modelo_inpe.split("_")[-3]
    separa = aux[0:12]
    data_modelo_inpe = dt.datetime.strptime(separa, '%Y%m%d%H%M')
    return data_modelo_inpe
# script para substituir os valores de temp 
def substitui_temps (arquivo_inpe, arquivo_media, nome_media):
    aux_tam = len(arquivo_media)
    #aux_tam = len(arquivo_inpe)
    arquivo_novo = pd.DataFrame(arquivo_inpe)
    arquivo_media = pd.DataFrame(arquivo_media)
    arquivo_media.fillna(-9999.99,inplace= True)
    for i in range(0, aux_tam, 1):  #aux_tam
        #buscando lat e lon nos arquivos de media 
        busca_lat = arquivo_media['lat'][i] 
        busca_lon = arquivo_media['lon'][i]

        # validando o valor de temp da media
        if ((arquivo_media[nome_media][i]  == '--') or (arquivo_media[nome_media][i]  == -9999.99 )):
            busca_temp = " "
        else :
            busca_temp = (arquivo_media[nome_media][i])
            #print(arquivo_media[nome_media][i])
        
        # pegando linha (indice) no arquivo onde tem a lat e long igual ao do arquivo com medias
        linha = arquivo_novo.index[(arquivo_novo['lat'] == busca_lat) & (arquivo_novo['lon'] == busca_lon)]
        linha2 = int(linha[0])
        
        # VERIFICANDO O BUSCA TEMP É NAN 
        if(busca_temp == " " ):
           
           continue 
        else:
           #arquivo_novo.loc[linha2, 'temp'] = ('{:.2f}'.format(busca_temp)) 
           busca_temp = pd.to_numeric(busca_temp) 
           aux = arquivo_novo.loc[linha2, 'press']
           arquivo_novo.loc[linha2, 'press'] = (busca_temp).round(2)
           
           #print(f'Alteracao feita em {aux} por {busca_temp.round(2)},  lat {busca_lat}, lon {busca_lon} linha {i} de {aux_tam}' )
  
        #arquivo_novo.loc[linha2, 'temp'] =['{:.2f}'.format(busca_temp)] 
        #arquivo_novo['temp'][linha2] = busca_temp
        
    return arquivo_novo
def pegando_8linhas(modelo_inpe):
# reading the header
    with open(modelo_inpe, 'rt') as infile:
        header = [next(infile) for x in range(8)]

#         satimages = header[0:4]
#         keys = header[7].split(' ')
    return header
def reabre_arquivos(caminho_salvar, titulo):
    nome = caminho_salvar + titulo
    with open(nome, 'rt') as file2:
        dado_medias = file2.readlines()
    return dado_medias
def merge_arquivos(linhas_iniciais, arquivo2, caminho_salvar, titulo):
    #pegando caminho do arquivo
    caminho_arquivo_destino = caminho_salvar + titulo
    
    # fazendo o join dos arquivos
    arquivo_final = linhas_iniciais + arquivo2
#     arquivo_final_str = '\n'.join(arquivo_final)

      # Removendo linhas em branco
    arquivo_final_sem_linhas_em_branco = [linha for linha in arquivo_final if linha.strip()]

    # Unindo as linhas novamente em uma única string
#     arquivo_final_str = '\n'.join(arquivo_final_sem_linhas_em_branco)
    arquivo_final_str = ''.join(arquivo_final_sem_linhas_em_branco)
    
# Salva o conteúdo no arquivo de destino
    with open(caminho_arquivo_destino, 'w') as destino:
        destino.write(arquivo_final_str)
def retorna_lista(caminho_do_arquivo):

    lista = []
    for nome_arquivo in os.listdir(caminho_do_arquivo):
        aux = os.path.join(caminho_do_arquivo, nome_arquivo)
        lista.append(aux)
    lista = sorted(lista)
    return lista

#==================================================================
#==================================================================
#  ================== FIMM ==================
#==================================================================
#==================================================================

In [96]:
# ======= CAMINHOS DOS ARQUIVOS DE ENTRADA #=============================================================================

#caminho_csv_INPE = '/home/leonardo/Área de Trabalho/MEGA/SCRIPTS PYTHON/SCRIPTS PYTHON/Projetos-python/AMVS/DADOS/REPROJ E EXTRA/CSV AMV INPE/'
caminho_csv_INPE = '/home/leonardo/Área de Trabalho/MEGA/SCRIPTS PYTHON/SCRIPTS PYTHON/Projetos-python/AMVS/DADOS/REPROJ E EXTRA/SAIDAS MODELO 2022/2022 restante/CSV INPE/'

#caminho_csv_media = '/home/leonardo/Área de Trabalho/MEGA/SCRIPTS PYTHON/SCRIPTS PYTHON/Projetos-python/AMVS/DADOS/REPROJ E EXTRA/CSV SAIDA/'
caminho_csv_media = '/home/leonardo/Área de Trabalho/MEGA/SCRIPTS PYTHON/SCRIPTS PYTHON/Projetos-python/AMVS/DADOS/REPROJ E EXTRA/SAIDAS MODELO 2022/2022 restante/CSV SAIDA/'

# CAMINHO PC NOVO
#caminho_csv_media = '/home/vlsantos/MEGA/SCRIPTS PYTHON/SCRIPTS PYTHON/Projetos-python/AMVS/DADOS/REPROJ E EXTRA/CSV SAIDA/'
#caminho_csv_INPE = '/home/vlsantos/MEGA/SCRIPTS PYTHON/SCRIPTS PYTHON/Projetos-python/AMVS/DADOS/REPROJ E EXTRA/CSV AMV INPE/'
#NAO ESQUECER DE SEMPRE MUDAR A ULTIMA PASTA ""

# ========================================================================================
modelo_inpe = retorna_lista(caminho_csv_INPE)
media_gerada= retorna_lista(caminho_csv_media)
# ========================================================================================

In [97]:
# ============ NOME DOS ARQUIVOS ==========================
# =========================================================

#lat,lon,press,Press_media_frio_05%,Press_media_frio_07%,Press_media_frio_10%,Press_media_frio_15%,Press_media_frio_20%,Press_media_frio_25%,media_vetor_19x19,desvio_padrao,Temp_AMV_INPE,PRESS_SAT,qi

# =========================================================
# =========================================================

In [98]:
#  RODANDO O SCRIPT --- CHAMA AS FUNCOES 
# ---- PC DE CASA
#caminho_salvar = '/media/leonardo/Leonardo/DADOS/DADOS PROPOSTA INPE/DADOS PARA ENTRADA NO MODELO RENATO/YAKECAN_15A20_05_2022/EDITADO 05 QUENTE/'
# ----------- PC DO INPE 

caminho_salvar = '/home/leonardo/Área de Trabalho/MEGA/SCRIPTS PYTHON/SCRIPTS PYTHON/Projetos-python/AMVS/DADOS/REPROJ E EXTRA/SAIDAS MODELO 2022/2022 restante/ARQUIVO MODIFICADO/'

#caminho_salvar = '/home/leonardo/Área de Trabalho/MEGA/SCRIPTS PYTHON/SCRIPTS PYTHON/Projetos-python/AMVS/DADOS/REPROJ E EXTRA/ARQ MODIFICADO/'
#caminho_salvar = '/home/vlsantos/MEGA/SCRIPTS PYTHON/SCRIPTS PYTHON/Projetos-python/AMVS/DADOS/REPROJ E EXTRA/ARQ MODIFICADO/'

busca_data(modelo_inpe, media_gerada, caminho_salvar, 'media_vetor_19x19')

2022-02-01 09:00:00 2022-02-01 09:00:00

S10635312_202202010900.nc.bin_2em1_qi, salvo em /home/leonardo/Área de Trabalho/MEGA/SCRIPTS PYTHON/SCRIPTS PYTHON/Projetos-python/AMVS/DADOS/REPROJ E EXTRA/SAIDAS MODELO 2022/2022 restante/ARQUIVO MODIFICADO/
2022-02-01 12:00:00 2022-02-01 12:00:00

S10635312_202202011200.nc.bin_2em1_qi, salvo em /home/leonardo/Área de Trabalho/MEGA/SCRIPTS PYTHON/SCRIPTS PYTHON/Projetos-python/AMVS/DADOS/REPROJ E EXTRA/SAIDAS MODELO 2022/2022 restante/ARQUIVO MODIFICADO/
2022-02-01 15:00:00 2022-02-01 15:00:00

S10635312_202202011500.nc.bin_2em1_qi, salvo em /home/leonardo/Área de Trabalho/MEGA/SCRIPTS PYTHON/SCRIPTS PYTHON/Projetos-python/AMVS/DADOS/REPROJ E EXTRA/SAIDAS MODELO 2022/2022 restante/ARQUIVO MODIFICADO/
2022-02-01 18:00:00 2022-02-01 18:00:00

S10635312_202202011800.nc.bin_2em1_qi, salvo em /home/leonardo/Área de Trabalho/MEGA/SCRIPTS PYTHON/SCRIPTS PYTHON/Projetos-python/AMVS/DADOS/REPROJ E EXTRA/SAIDAS MODELO 2022/2022 restante/ARQUIVO MODIFICA

In [99]:
#==========================================================================
#==========================================================================
#==========================================================================
#==========================================================================
#================= =======  Area de teste dos scripts ============== ==== =
# ================= ================= ================= ================= =
#==========================================================================

In [100]:
# testando data

# TESTANDO DATA 

#modelo_inpe[1]
#data  = retorna_data_modelo_inpe(modelo_inpe[1])
#str(data)

#TESTANDO DATA DA  MEDIA DO AMV
#media_gerada[0]
#
#data_media = retorna_data__media_gerada(media_gerada[0])
#print(data_media)
#aux = media_gerada[0].split('_')[-3]
#ano= int(aux[1:5])
#dj = int(aux[5:8])
#hora= int(aux[8:10])

#data_amv = datetime.strptime(f"{ano}-{dj}", "%Y-%j")  # Convertendo ano e dia juliano
#data_amv = data_amv.replace(hour=hora)


##def substitui_temps1 (arquivo_inpe, arquivo_media, nome_media):
#    aux_tam = len(arquivo_media)
#    #aux_tam = len(arquivo_inpe)
#    arquivo_novo = pd.DataFrame(arquivo_inpe)
#    arquivo_media = pd.DataFrame(arquivo_media)
#    
#    for i in range(0, aux_tam, 1):  #aux_tam
#        #buscando lat e lon nos arquivos de media 
#        busca_lat = arquivo_media['lat'][i] 
#        busca_lon = arquivo_media['lon'][i]
#
#        # validando o valor de temp da media
#        if (arquivo_media[nome_media][i]  == '--'):
#            busca_temp = " "
#        else :
#            busca_temp = (arquivo_media[nome_media][i])
#            #print(arquivo_media[nome_media][i])
#        
#        # pegando linha (indice) no arquivo onde tem a lat e long igual ao do arquivo com medias
#        linha = arquivo_novo.index[(arquivo_novo['lat'] == busca_lat) & (arquivo_novo['lon'] == busca_lon)]
#        linha2 = int(linha[0])
#        
#        # VERIFICANDO O BUSCA TEMP É NAN 
#        if(busca_temp == " " ):
#           
#           continue 
#        else:
#           #arquivo_novo.loc[linha2, 'temp'] = ('{:.2f}'.format(busca_temp)) 
#           busca_temp = pd.to_numeric(busca_temp) 
#           aux = arquivo_novo.loc[linha2, 'press']
#           arquivo_novo.loc[linha2, 'press'] = (busca_temp).round(2)
#           
#           print(f'Alteracao feita em {aux} por {busca_temp.round(2)},  lat {busca_lat}, lon {busca_lon} linha {i} de {aux_tam}' )
#  
#        #arquivo_novo.loc[linha2, 'temp'] =['{:.2f}'.format(busca_temp)] 
#        #arquivo_novo['temp'][linha2] = busca_temp
#        
#    return arquivo_novo

In [101]:
##  TESTANDO O CAMINHO DO ARQUIVO E O PROBLEMA DE COLUNAS
#caminho_teste = '/home/leonardo/Área de Trabalho/MEGA/SCRIPTS PYTHON/SCRIPTS PYTHON/Projetos-python/AMVS/DADOS/REPROJ E EXTRA/ARQ MODIFICADO/'
#nome_original = 'S10635312_202106271200.nc.bin_2em1_qi'
###nome_editado = 'OR_ABI-L2-ACHTF-M6_G16_s20221350000202_e20221350009510_c20221350013136.nc_ret.nc_ret.csv'
##nome_arquivo_trocado = 'S10635312_202205150000.nc.bin_2em1_qi'
##
#cam_arquivo_original = caminho_teste + nome_original
##cam_arquivo_editado = caminho_teste + nome_editado
##cam_arquivo_trocado = caminho_teste + nome_arquivo_trocado   # arquivo que fiz a substituiçao de temps
##
# # LENDO O ARquivo
##with open (cam_completo, 'r') as arquivo:
##     arquivo2 = arquivo.readlines()
##
##    # CONTANDO NUMERO DE COLUNAS E VENDO SE ESTA DIFERENTE
#with open (cam_arquivo_original, 'r') as file:
#    original = file.readlines()
##
#cont = 0
#for i, line in enumerate(original):
##    
#    num_columns = len(line.split())
#    if num_columns != 41 :
#        print(i, num_columns)
#        cont = cont+1
##            
##tam = len(original)
#tam, cont
## type(arquivo2)

In [102]:
# LINHA PROBLEMATICA E VERIFICANDO QUAL O ERRO
#linha_problematica = original[5051]
#print(linha_problematica)

In [103]:
##ABRINDO DADOS DAS MEDIAS QUE CALCULEI
#dado_medias = pd.read_csv(cam_arquivo_trocado,sep = ' ',skiprows = 7,                         
#                        header = 0, encoding = 'latin1'  )
#
##dado_medias.head().T
##dado_medias.info()
#linha = dado_medias.iloc[12335] #13693
##linha.info()
#
##print(linha['lat'])
##print(linha['lon'])
##print(linha)
#
#dado_medias_feitas = pd.read_csv(cam_arquivo_editado,sep = ',',skiprows = 0,                         
#                        header = 0, encoding = 'latin1'  )
#
#busca_lat = linha['lat']
#busca_lon = linha['lon']
##print(busca_lon, busca_lat)
#
#conf = dado_medias_feitas.index[(dado_medias_feitas['lat'] == busca_lat) & (dado_medias_feitas['lon'] == busca_lon)]
#
#linha2 = int(conf[0])
##print(linha2)
#dado_medias_feitas.iloc[int(linha2)] # media_frio_05%    210.892868
#
##print(linha['temp'])
#linha 
